# Run Inference on Multi-Region S3 Buckets

### Prerequisites
- terraform
- python
- aws cli (configured)

In [ ]:
%cd multi-region

## Add your regions to regions.md
### Note: The Bootstrap region should be first

In [2]:
%cat regions.md

# aws ec2 describe-regions | jq -r '.Regions[].RegionName'
# Bootstrap region must be first
ca-central-1
eu-north-1
us-east-1
us-west-1


In [5]:
# install dependencies
%pip install boto3 ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 10.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Replace the value of the name parameter with the name of your AMI
%python3 get_instance_amis.py --name "Deep Learning AMI GPU PyTorch 2.0.1 (Ubuntu 20.04) 20230620"

{
    "ca-central-1": {
        "region": "ca-central-1",
        "zone": "ca-central-1a",
        "instance_ami": "ami-0c483e1cc1d6e27f8"
    },
    "eu-north-1": {
        "region": "eu-north-1",
        "zone": "eu-north-1a",
        "instance_ami": "ami-033855fe5b2db3d01"
    },
    "us-east-1": {
        "region": "us-east-1",
        "zone": "us-east-1a",
        "instance_ami": "ami-0240b7264c1c9e6a9"
    },
    "us-west-1": {
        "region": "us-west-1",
        "zone": "us-west-1a",
        "instance_ami": "ami-0eaef2301ab069ace"
    }
}


## Replace the Values of "locations" with the values outputted from the above command

In [10]:
%cat ./tf/.env.example

{
  "access_key": "AKIAUEMPQ7JFEXAMPLE",
  "secret_key": "a3YhVb06muc4+YSnlzqLzJJ2sOrxl/EXAMPLE",
  "bootstrap_region": "ca-central-1", # Bootstrap region must be firts (also, delete this line in the final, json doesn't support comments)
  "locations": {
    "ca-central-1": {
      "region": "ca-central-1",
      "availability_zone": "ca-central-1a",
      "instance_ami": "ami-0ea18256de20ecdfc"
    },
    "us-west-2": {
      "region": "us-west-2",
      "availability_zone": "us-west-2a",
      "instance_ami": "ami-0fcf52bcf5db7b003"
    },
    "eu-west-1": {
      "region": "eu-west-1",
      "availability_zone": "eu-west-1a",
      "instance_ami": "ami-00aa9d3df94c6c354"
    },
    "eu-north-1": {
      "region": "eu-north-1",
      "availability_zone": "eu-north-1a",
      "instance_ami": "ami-0a79730daaf45078a"
    },
    "ap-southeast-1": {
      "region": "ap-southeast-1",
      "availability_zone": "ap-southeast-1a",
      "instance_ami": "ami-0a72af05d27b49ccb"
    }
  },
  "a

## Replace The values of the keys with your own key values and also the value of instance_type if you want

In [ ]:
%%bash
cp -r .env.example .env.json

### Deploying the infra

In [ ]:
%%bash
./bulk-deploy.sh create

After successfully completing the Terraform deployment, ensure you've set up the necessary environment variables. Execute the following command in your terminal:

```
source tf/aws/baclhau.run
REGION=$(awk '!/^#/' regions.md | head -n 1)
export BACALHAU_NODE_CLIENTAPI_HOST=$(jq -r '.outputs.ip_address.value' "./tf/aws/terraform.tfstate.d/${REGION}/terraform.tfstate")
```

### Populating the Buckets with random samples from the coco-2017 val dataset
Putting 10 sample images in each bucket

In [ ]:
%%bash
python3 populate_buckets.py --sample_size 10

### Running the job to get all the images with a person in them

You can replace "person" with the term you want

In [ ]:
%%bash
python3 job.py "person"

### Downloading the outputs

In [ ]:
%%bash
python3 download_outputs.py

### Viewing the outputs

In [1]:
import os
from PIL import Image
import ipywidgets as widgets
from IPython.display import display

# Path to the folder where the images are stored
image_folder = os.path.join(os.getcwd(), 'outputs')

# List to store the images
images = []

# Go through all the files in the folder
for filename in os.listdir(image_folder):
    # Check if the file is an image
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Open the image and append it to the list
        images.append(Image.open(os.path.join(image_folder, filename)))

# Create a slider
slider = widgets.IntSlider(min=0, max=len(images)-1, step=1)

# Function to display an image
def display_image(i):
    display(images[i])

# Link the slider to the function
widgets.interact(display_image, i=slider)

interactive(children=(IntSlider(value=0, description='i', max=9), Output()), _dom_classes=('widget-interact',)…

<function __main__.display_image(i)>